In [95]:
import re
from collections import Counter

def load(file):
    with open(file, encoding="utf-8") as f:
        return [l.strip().lower() for l in f if l.strip()]

words = set(load("en.txt") + load("bad-words.txt")*10000)

# filter
clean = [
    w for w in words
    if len(w) > 3 and re.fullmatch(r"[a-z]+", w)
]

IMPORTANT = {
    "fuck": 1000,
}

with open("sp_train.txt", "w", encoding="utf-8") as f:
    for w in clean:
        boost = IMPORTANT.get(w, 150)
        f.write((w + " ") * boost + "\n")


In [96]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(
    input="sp_train.txt",
    model_prefix="custom_sp",
    vocab_size=3000,
    model_type="bpe",
    character_coverage=1.0,
    split_by_whitespace=True
)


In [97]:
sp = spm.SentencePieceProcessor()
sp.load("custom_sp.model")

tests = [
    "killforyou",
    "fuckyour mother",
    "sisterfucker",
    "killeritems",
    'suckmydick',
    'motherfuckerjews',
    'xgspotforyou'
]

for t in tests:
    print(t, "->", sp.encode(t, out_type=str))


killforyou -> ['▁kill', 'f', 'or', 'you']
fuckyour mother -> ['▁fuck', 'y', 'our', '▁mother']
sisterfucker -> ['▁s', 'ist', 'er', 'fucker']
killeritems -> ['▁killer', 'it', 'em', 's']
suckmydick -> ['▁suckmy', 'dick']
motherfuckerjews -> ['▁mother', 'fucker', 'j', 'ew', 's']
xgspotforyou -> ['▁', 'x', 'g', 'spot', 'f', 'or', 'you']
